# **Chicago Car Crashes**

# Overview

# Business Problem

Build a classifier to predict the primary contributory cause of a car accident, given information about the car, the people in the car, the road conditions etc. You might imagine your audience as a Vehicle Safety Board who's interested in reducing traffic accidents, or as the City of Chicago who's interested in becoming aware of any interesting patterns.

This is a multi-class classification problem. You will almost certainly want to bin, trim or otherwise limit the number of target categories on which you ultimately predict. Note that some primary contributory causes have very few samples, for example.

# Data Understanding


In [3]:
# Importing packages
import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.dummy import DummyClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import roc_curve, roc_auc_score, RocCurveDisplay
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# RocCurveDisplay.from_estimator(logreg, X_test_scaled, y_test)

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Read in Crashes Data
df_crashes = pd.read_csv("data/Traffic_Crashes_-_Crashes_20231109.csv")

In [5]:
# Read in Vehicles Data
df_vehicles = pd.read_csv("data/Traffic_Crashes_-_Vehicles_20231109.csv", low_memory=False)

In [6]:
# Read in People Data
df_people = pd.read_csv("data/Traffic_Crashes_-_People_20231109.csv", low_memory=False)

# EDA

In [7]:
df_crashes.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,JG387648,NaN,08/18/2023 12:50:00 PM,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,1.0,0.0,1.0,0.0,12,6,8,NaN,NaN,NaN
1,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,JG361138,NaN,07/29/2023 02:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,0.0,0.0,1.0,0.0,14,7,7,41.854120,-87.665902,POINT (-87.665902342962 41.854120262952)
2,61fcb8c1eb522a6469b460e2134df3d15f82e81fd93e9c...,JG388133,NaN,08/18/2023 05:58:00 PM,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDALCYCLIST,...,1.0,0.0,1.0,0.0,17,6,8,41.942976,-87.761883,POINT (-87.761883496974 41.942975745006)
3,004cd14d0303a9163aad69a2d7f341b7da2a8572b2ab33...,JC524901,NaN,11/26/2019 08:38:00 AM,25,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDESTRIAN,...,0.0,0.0,1.0,0.0,8,3,11,NaN,NaN,NaN
4,a1d5f0ea90897745365a4cbb06cc60329a120d89753fac...,JG387475,NaN,08/18/2023 10:45:00 AM,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,FIXED OBJECT,...,0.0,0.0,1.0,0.0,10,6,8,NaN,NaN,NaN


In [8]:
df_people.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,NaN,CHICAGO,IL,60651,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,NaN,CHICAGO,IL,60620,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,IMPROPER BACKING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,NaN,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [9]:
df_vehicles.head()

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,1554880,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,04/14/2023 02:05:00 PM,1,DRIVER,NaN,1478881.0,NaN,FORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2,PARKED,NaN,834819.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1554881,91a5d08b2b701f2d37cbb52ecdbeb09579bc7f2ebc60b3...,JG223284,04/14/2023 02:05:00 PM,2,DRIVER,1.0,1478892.0,NaN,ISUZU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1588937 entries, 0 to 1588936
Data columns (total 72 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH_UNIT_ID             1588937 non-null  int64  
 1   CRASH_RECORD_ID           1588937 non-null  object 
 2   RD_NO                     1579642 non-null  object 
 3   CRASH_DATE                1588937 non-null  object 
 4   UNIT_NO                   1588937 non-null  int64  
 5   UNIT_TYPE                 1586855 non-null  object 
 6   NUM_PASSENGERS            235108 non-null   float64
 7   VEHICLE_ID                1552556 non-null  float64
 8   CMRC_VEH_I                29662 non-null    object 
 9   MAKE                      1552551 non-null  object 
 10  MODEL                     1552406 non-null  object 
 11  LIC_PLATE_STATE           1410532 non-null  object 
 12  VEHICLE_YEAR              1301948 non-null  float64
 13  VEHICLE_DEFECT            1

In [11]:
df_vehicles["CRASH_DATE"].value_counts()

CRASH_DATE
12/29/2020 05:00:00 PM    60
11/10/2017 10:30:00 AM    58
02/17/2022 03:30:00 PM    42
11/10/2017 10:00:00 AM    42
01/12/2019 02:30:00 PM    41
                          ..
11/19/2019 02:23:00 PM     1
12/27/2021 10:40:00 AM     1
12/27/2021 10:03:00 AM     1
11/19/2019 03:39:00 PM     1
07/30/2021 01:30:00 AM     1
Name: count, Length: 510079, dtype: int64

In [12]:
df_vehicles.columns

Index(['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE', 'UNIT_NO',
       'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE',
       'MODEL', 'LIC_PLATE_STATE', 'VEHICLE_YEAR', 'VEHICLE_DEFECT',
       'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION', 'MANEUVER',
       'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY',
       'TOWED_TO', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I',
       'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I',
       'AREA_09_I', 'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I',
       'FIRST_CONTACT_POINT', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO',
       'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE',
       'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO',
       'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO',
       'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I',
       'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I',
       '

In [ ]:
## Change column names to lowercase for readability
# df_vehicles.columns = df_vehicles.columns.str.lower()

In [13]:
# EDA crashes data
df_crashes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778847 entries, 0 to 778846
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                778847 non-null  object 
 1   RD_NO                          774292 non-null  object 
 2   CRASH_DATE_EST_I               58506 non-null   object 
 3   CRASH_DATE                     778847 non-null  object 
 4   POSTED_SPEED_LIMIT             778847 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         778847 non-null  object 
 6   DEVICE_CONDITION               778847 non-null  object 
 7   WEATHER_CONDITION              778847 non-null  object 
 8   LIGHTING_CONDITION             778847 non-null  object 
 9   FIRST_CRASH_TYPE               778847 non-null  object 
 10  TRAFFICWAY_TYPE                778847 non-null  object 
 11  LANE_CNT                       199006 non-null  float64
 12  ALIGNMENT                     

In [14]:
# Exploring date column - Dec 29th stands out
df_crashes["CRASH_DATE"].value_counts()

CRASH_DATE
12/29/2020 05:00:00 PM    30
11/10/2017 10:30:00 AM    27
02/17/2022 03:30:00 PM    21
01/12/2019 02:30:00 PM    20
11/10/2017 10:00:00 AM    20
                          ..
08/11/2020 01:28:00 PM     1
03/12/2018 01:08:00 PM     1
11/20/2018 06:41:00 PM     1
06/08/2018 02:31:00 AM     1
10/18/2020 12:03:00 AM     1
Name: count, Length: 510053, dtype: int64

In [39]:
# Convert "CRASH_DATE" to datetime data type
df_crashes["CRASH_DATE"] = pd.to_datetime(df_crashes["CRASH_DATE"])

years = df_crashes[(df_crashes['CRASH_DATE'].dt.year >= 2020) & (df_crashes['CRASH_DATE'].dt.year <= 2022)]
years


,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,Month,Year
9,359bf9f5872d646bb63576e55b1e0b480dc93c2b935ab5...,JF130143,NaN,2022-01-31 19:45:00,25,NO CONTROLS,NO CONTROLS,CLEAR,DARKNESS,REAR END,...,5.0,0.0,19,2,1,NaN,NaN,NaN,1,2022
10,36360857c079418cba1b1d70cf653595bbfb4566de8fcb...,JF100705,Y,2022-01-01 16:32:00,10,NO CONTROLS,NO CONTROLS,SNOW,"DARKNESS, LIGHTED ROAD",ANGLE,...,2.0,0.0,16,7,1,NaN,NaN,NaN,1,2022
18,37a215843a67b9d2118972242e0ab68232583ffe20401f...,JD403481,NaN,2020-10-18 15:58:00,35,NO CONTROLS,NO CONTROLS,RAIN,DAYLIGHT,FIXED OBJECT,...,1.0,0.0,15,1,10,NaN,NaN,NaN,10,2020
19,3b6d23138e3f009e54ae5ef25061e9e96fe44fd0e62afa...,JE471264,NaN,2021-12-09 10:30:00,25,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,3.0,0.0,10,5,12,NaN,NaN,NaN,12,2021
27,456dd9d9bc16f505f93bd2ef89b174348c36dadf1fdc0a...,JE241280,NaN,2021-05-24 15:35:00,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,1.0,0.0,15,2,5,NaN,NaN,NaN,5,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778833,fcf3c5ddd8ba79a1b43c039a2507b201c999d659aa85d3...,JD207407,NaN,2020-04-12 07:10:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,...,2.0,0.0,7,1,4,41.847421,-87.612391,POINT (-87.61239120397 41.847420982326),4,2020
778835,e38400c0474e70f5aa9f0e0f115957281ff234b6727e00...,JD213164,NaN,2020-04-20 15:30:00,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,...,2.0,0.0,15,2,4,41.740031,-87.629704,POINT (-87.629704407703 41.74003058451),4,2020
778837,fb93c3183712074555fc2062c807c10dc6e25be439705b...,JD244997,NaN,2020-05-28 00:12:00,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,RAIN,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,...,1.0,0.0,0,5,5,41.920583,-87.737606,POINT (-87.737605858165 41.920582618046),5,2020
778839,6bd73dc0b10ee0272e9269bec32b57327611c2ad3cf19b...,JE435503,NaN,2021-11-05 22:56:00,30,OTHER,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",OTHER OBJECT,...,1.0,0.0,22,6,11,41.876044,-87.700459,POINT (-87.700459346481 41.876043663599),11,2021


In [19]:
# Convert "CRASH_DATE" to datetime data type
df_crashes["CRASH_DATE"] = pd.to_datetime(df_crashes["CRASH_DATE"])

# Extract the month from "CRASH_DATE"
df_crashes["Month"] = df_crashes["CRASH_DATE"].dt.month

# Count occurrences of each month
month_counts = df_crashes["Month"].value_counts()

# Identify the month with the highest count
most_common_month = month_counts.idxmax()
most_common_month

10

In [20]:
# The day of the week component of CRASH_DATE. Sunday=1
df_crashes["CRASH_DAY_OF_WEEK"].value_counts()

CRASH_DAY_OF_WEEK
6    126522
7    115291
5    111692
3    111060
4    110361
2    107243
1     96678
Name: count, dtype: int64

In [21]:
# Total persons sustaining fatal, incapacitating, non-incapacitating, and possible injuries as determined by the reporting officer
injuries_counts = df_crashes["INJURIES_TOTAL"].value_counts()
injuries_counts

INJURIES_TOTAL
0.0     670544
1.0      80305
2.0      17802
3.0       5393
4.0       1929
5.0        683
6.0        274
7.0        104
8.0         43
9.0         24
10.0        14
11.0         8
15.0         7
21.0         4
12.0         4
13.0         2
17.0         1
14.0         1
19.0         1
16.0         1
Name: count, dtype: int64

In [22]:
# Fatal injuries
injuries_fatal = df_crashes["INJURIES_FATAL"].value_counts()
injuries_fatal

INJURIES_FATAL
0.0    776287
1.0       796
2.0        52
3.0         8
4.0         1
Name: count, dtype: int64

In [23]:
df_crashes["CRASH_TYPE"].value_counts()

CRASH_TYPE
NO INJURY / DRIVE AWAY              571926
INJURY AND / OR TOW DUE TO CRASH    206921
Name: count, dtype: int64

In [24]:
df_crashes["FIRST_CRASH_TYPE"].value_counts()

FIRST_CRASH_TYPE
PARKED MOTOR VEHICLE            181193
REAR END                        175239
SIDESWIPE SAME DIRECTION        118693
TURNING                         110954
ANGLE                            84304
FIXED OBJECT                     36601
PEDESTRIAN                       17941
PEDALCYCLIST                     11880
SIDESWIPE OPPOSITE DIRECTION     11020
OTHER OBJECT                      7729
REAR TO FRONT                     7257
HEAD ON                           6650
REAR TO SIDE                      4312
OTHER NONCOLLISION                2486
REAR TO REAR                      1516
ANIMAL                             553
OVERTURNED                         477
TRAIN                               42
Name: count, dtype: int64

In [25]:
df_crashes["PRIM_CONTRIBUTORY_CAUSE"].value_counts()

PRIM_CONTRIBUTORY_CAUSE
UNABLE TO DETERMINE                                                                 302153
FAILING TO YIELD RIGHT-OF-WAY                                                        85315
FOLLOWING TOO CLOSELY                                                                76341
NOT APPLICABLE                                                                       41267
IMPROPER OVERTAKING/PASSING                                                          38280
FAILING TO REDUCE SPEED TO AVOID CRASH                                               32933
IMPROPER BACKING                                                                     30962
IMPROPER LANE USAGE                                                                  28112
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  25874
IMPROPER TURNING/NO SIGNAL                                                           25859
DISREGARDING TRAFFIC SIGNALS                                      

In [26]:
# Filtering by removing Unable to determine cause
# Total decreases from 778437 to 476K 
df_crashes[df_crashes["PRIM_CONTRIBUTORY_CAUSE"] != "UNABLE TO DETERMINE"]


,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,...,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,Month
0,6c1659069e9c6285a650e70d6f9b574ed5f64c12888479...,JG387648,NaN,2023-08-18 12:50:00,15,OTHER,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,1.0,0.0,12,6,8,NaN,NaN,NaN,8
1,5f54a59fcb087b12ae5b1acff96a3caf4f2d37e79f8db4...,JG361138,NaN,2023-07-29 14:45:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,PARKED MOTOR VEHICLE,...,0.0,1.0,0.0,14,7,7,41.854120,-87.665902,POINT (-87.665902342962 41.854120262952),7
2,61fcb8c1eb522a6469b460e2134df3d15f82e81fd93e9c...,JG388133,NaN,2023-08-18 17:58:00,30,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,PEDALCYCLIST,...,0.0,1.0,0.0,17,6,8,41.942976,-87.761883,POINT (-87.761883496974 41.942975745006),8
4,a1d5f0ea90897745365a4cbb06cc60329a120d89753fac...,JG387475,NaN,2023-08-18 10:45:00,20,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,FIXED OBJECT,...,0.0,1.0,0.0,10,6,8,NaN,NaN,NaN,8
7,0e208d23344f0d1b3a9fcd4bb07676a750ddb73c397b5c...,JG381551,NaN,2023-08-13 13:30:00,35,NO CONTROLS,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,ANGLE,...,0.0,2.0,0.0,13,1,8,NaN,NaN,NaN,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778835,e38400c0474e70f5aa9f0e0f115957281ff234b6727e00...,JD213164,NaN,2020-04-20 15:30:00,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,TURNING,...,0.0,2.0,0.0,15,2,4,41.740031,-87.629704,POINT (-87.629704407703 41.74003058451),4
778838,89dc61af34d393db950397f0cc06d53b56d1f5e5fa14d4...,JG317141,NaN,2023-06-26 16:50:00,35,NO CONTROLS,NO CONTROLS,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,...,0.0,4.0,0.0,16,2,6,41.746905,-87.644077,POINT (-87.644077151581 41.746904607442),6
778840,3d00cf22a912d0e18809db862dd67e5812f7b6af1ffa3d...,JG346351,NaN,2023-07-18 14:10:00,30,UNKNOWN,UNKNOWN,CLEAR,DAYLIGHT,SIDESWIPE SAME DIRECTION,...,0.0,2.0,0.0,14,3,7,41.778580,-87.738679,POINT (-87.738679437114 41.77857996073),7
778842,6dee8823d4ae96624b741428681d19f50b5960418b6d79...,JG408597,NaN,2023-09-02 18:25:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,...,0.0,2.0,0.0,18,7,9,41.758092,-87.624902,POINT (-87.624902228247 41.758092176383),9


In [27]:
df_crashes['WEATHER_CONDITION'].value_counts()

WEATHER_CONDITION
CLEAR                       613562
RAIN                         67442
UNKNOWN                      41911
SNOW                         26623
CLOUDY/OVERCAST              22893
OTHER                         2404
FREEZING RAIN/DRIZZLE         1407
FOG/SMOKE/HAZE                1110
SLEET/HAIL                     943
BLOWING SNOW                   398
SEVERE CROSS WIND GATE         147
BLOWING SAND, SOIL, DIRT         7
Name: count, dtype: int64

In [41]:
# columns to drop
#df_crashes.drop( rd_no, crash_date_est_i, report_type, data_police-Notified, sec_contributory_cause, street_no, street_direction, street_name, beat_of_occurence, photos_taken, statements_taken, work_zone_i, latitute, longitute, location  


CRASH_DATE_EST_I
Y    51001
N     7505
Name: count, dtype: int64

In [ ]:
# columns to drop
# df_vehicles (rd_no, unit_no, unit_type, vehicle_id, travel_direciton, towed_i, fire_i, occupant_count, towed_by, towed_to, area_all, cmv_id, usdot_not, ccmc_no, ilcc_no, commercial_src, gvwr, carrier_name, carrier_state, carrier_city, hazmat_placards_i, 
# hazmat_name, un_no, hazmat_present_i, hazmat_report_i, hazmat_report_no, mcs_report_all, drop the rest, keep total_vehicle_length and vehicle_config, drop cargo_body_type, ) trucks 

In [ ]:
# keep columns in people data 
# sex, age, safety_equipment, airbag, injury classfication, driver_action, vision, physical_condi, ped location, bac, cell use

In [28]:
# Exploring Vehicles Dataset
df_vehicles["CRASH_DATE"].value_counts()

CRASH_DATE
12/29/2020 05:00:00 PM    60
11/10/2017 10:30:00 AM    58
02/17/2022 03:30:00 PM    42
11/10/2017 10:00:00 AM    42
01/12/2019 02:30:00 PM    41
                          ..
11/19/2019 02:23:00 PM     1
12/27/2021 10:40:00 AM     1
12/27/2021 10:03:00 AM     1
11/19/2019 03:39:00 PM     1
07/30/2021 01:30:00 AM     1
Name: count, Length: 510079, dtype: int64

In [29]:
df_vehicles["CMRC_VEH_I"].value_counts()

CMRC_VEH_I
Y    18958
N    10704
Name: count, dtype: int64

In [30]:
# Exploring the People Dataset
df_people['CRASH_DATE'].value_counts()

CRASH_DATE
12/29/2020 05:00:00 PM    72
11/10/2017 10:30:00 AM    64
03/16/2018 10:17:00 AM    61
06/22/2019 06:15:00 PM    55
07/12/2023 10:40:00 AM    50
                          ..
01/02/2018 08:52:00 PM     1
02/11/2020 12:10:00 AM     1
01/02/2018 08:28:00 PM     1
02/24/2021 06:39:00 PM     1
11/22/2015 01:58:00 PM     1
Name: count, Length: 509164, dtype: int64

In [31]:
# Converting CRASH_DATE to datetime to explore months and years of crashes

df_people['CRASH_DATE'] = pd.to_datetime(df_people['CRASH_DATE'])

# Create new columns for month and year
df_people['Month'] = df_people['CRASH_DATE'].dt.month
df_people['Year'] = df_people['CRASH_DATE'].dt.year

# Sort the DataFrame by year
sorted_df = df_people.sort_values(by=['Year'])

# Count the crashes for each year
crash_counts_by_year = sorted_df.groupby(['Year']).size().reset_index(name='Crash_Count')

# Print the result
print(crash_counts_by_year)


    Year  Crash_Count
0   2013            5
1   2014           11
2   2015        20933
3   2016        96022
4   2017       185328
5   2018       265709
6   2019       264007
7   2020       199983
8   2021       233794
9   2022       233819
10  2023       210467


In [32]:
# Driver action that contributed to the crash, as determined by reporting officer
df_people["DRIVER_ACTION"].value_counts()

DRIVER_ACTION
NONE                                 488224
UNKNOWN                              340884
FAILED TO YIELD                      123908
OTHER                                121181
FOLLOWED TOO CLOSELY                  81516
IMPROPER BACKING                      40766
IMPROPER TURN                         35662
IMPROPER LANE CHANGE                  35018
IMPROPER PASSING                      30330
DISREGARDED CONTROL DEVICES           23852
TOO FAST FOR CONDITIONS               20583
WRONG WAY/SIDE                         5441
IMPROPER PARKING                       5029
OVERCORRECTED                          2403
EVADING POLICE VEHICLE                 2152
CELL PHONE USE OTHER THAN TEXTING      2038
EMERGENCY VEHICLE ON CALL              1240
TEXTING                                 543
STOPPED SCHOOL BUS                      164
LICENSE RESTRICTIONS                     56
Name: count, dtype: int64

In [33]:
# Status of blood alcohol concentration testing for driver or other person involved in crash
df_people["BAC_RESULT"].value_counts()

BAC_RESULT
TEST NOT OFFERED                   1341864
TEST REFUSED                         14318
TEST PERFORMED, RESULTS UNKNOWN       3347
TEST TAKEN                            2440
Name: count, dtype: int64

In [34]:
# Whether person was/was not using cellphone at the time of the crash, as determined by the reporting officer
df_people["CELL_PHONE_USE"].value_counts()

CELL_PHONE_USE
Y    752
N    406
Name: count, dtype: int64

In [35]:
df_people["CELL_PHONE_USE"].isna().sum()

1708920

In [36]:
df_people["INJURY_CLASSIFICATION"].value_counts()

INJURY_CLASSIFICATION
NO INDICATION OF INJURY     1561840
NONINCAPACITATING INJURY      83359
REPORTED, NOT EVIDENT         47454
INCAPACITATING INJURY         15765
FATAL                           949
Name: count, dtype: int64